# Project loader

In [1]:
%pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 1.4 MB/s eta 0:00:01
   --- ------------------------------------ 0.1/1.2 MB 1.1 MB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.2 MB 1.3 MB/s eta 0:00:01
   -------- ------------------------------- 0.2/1.2 MB 1.4 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.2 MB 1.4 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.2 MB 1.4 MB/s eta 0:00:01
   --------- ------------------------------ 0.3/1.2 MB 896.4 kB/s eta 0:00:01
   ----------- ---------------------------- 0.3/1.2 MB 952.6 kB/s eta 0:00:01
   -------------- ------------------------- 0.4/1.2 MB 1.0 MB/s eta 0:00:01
   ---------------- ----------------------- 0.5/1.2 MB 1.1 MB/s eta 0:00:01
   ------------------- -------------------- 0.6/1.2 MB 1.1 MB/s eta 0:00:01
   -------------------- ------------------- 0.6/1.2 MB 1.1 MB/s eta 0:00:01
   ------------

In [1]:
import os
import psycopg2

Define database connection.

In [2]:
conn = psycopg2.connect (
    database="airbnb", 
    user='postgres', password='root',  
    host='127.0.0.1', port='5432'
)

Get the cursor.

In [3]:
conn.autocommit = True
cursor = conn.cursor() 

Retrieve full path for every csv file under out folder.

In [4]:
OUT_FOLDER_PATH = 'ETL/out/'

In [11]:
full_paths = [
        os.path.abspath(os.path.join(OUT_FOLDER_PATH, file)) 
        for file in os.listdir(OUT_FOLDER_PATH)
] 

In [12]:
full_paths

['c:\\Users\\giaco\\Desktop\\Progetti\\NYC_AirBnB_Data_Management\\ETL\\out\\listings.csv',
 'c:\\Users\\giaco\\Desktop\\Progetti\\NYC_AirBnB_Data_Management\\ETL\\out\\nypdarrests.csv',
 'c:\\Users\\giaco\\Desktop\\Progetti\\NYC_AirBnB_Data_Management\\ETL\\out\\rollingsales.csv',
 'c:\\Users\\giaco\\Desktop\\Progetti\\NYC_AirBnB_Data_Management\\ETL\\out\\subway_stops.csv']

Define the query that runs the `COPY` command. It will be responsible for loading the csv into the specified tables.

In [13]:
table_names = [
    'listings',
    'nypd_Arrests',
    'house_sales',
    'subway_stops'
]

In [15]:
for i, table in enumerate(table_names):
    with open(full_paths[i], 'r') as f:
        cursor.copy_expert(f'COPY {table} FROM STDIN WITH HEADER CSV', f)